## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,59.81,82,99,9.62,overcast clouds
1,1,Port Moresby,PG,-9.4431,147.1797,85.42,66,75,5.75,broken clouds
2,2,Provideniya,RU,64.3833,-173.3000,28.90,51,15,12.21,few clouds
3,3,Batagay-Alyta,RU,67.8006,130.4114,19.89,92,100,12.77,light snow
4,4,Pevek,RU,69.7008,170.3133,21.15,71,100,8.08,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Port Moresby,PG,-9.4431,147.1797,85.42,66,75,5.75,broken clouds
8,8,Hobyo,SO,5.3505,48.5268,83.79,71,100,16.49,light rain
15,15,Kapaa,US,22.0752,-159.3190,84.18,78,40,14.97,light rain
24,24,Cayenne,GF,4.9333,-52.3333,81.09,1,100,5.99,overcast clouds
30,30,Kathu,TH,7.9178,98.3332,85.82,99,100,6.73,overcast clouds
31,31,Rikitea,PF,-23.1203,-134.9692,76.46,71,100,5.53,overcast clouds
36,36,Atuona,PF,-9.8000,-139.0333,78.22,82,100,14.54,light rain
38,38,Ahmadnagar,IN,19.0833,74.7333,80.01,50,0,7.29,clear sky
45,45,Sorong,ID,-0.8833,131.2500,84.69,72,35,6.78,scattered clouds
51,51,Banjar,ID,-8.1900,114.9675,87.33,62,96,1.70,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Port Moresby,PG,85.42,broken clouds,-9.4431,147.1797,
8,Hobyo,SO,83.79,light rain,5.3505,48.5268,
15,Kapaa,US,84.18,light rain,22.0752,-159.3190,
24,Cayenne,GF,81.09,overcast clouds,4.9333,-52.3333,
30,Kathu,TH,85.82,overcast clouds,7.9178,98.3332,
31,Rikitea,PF,76.46,overcast clouds,-23.1203,-134.9692,
36,Atuona,PF,78.22,light rain,-9.8000,-139.0333,
38,Ahmadnagar,IN,80.01,clear sky,19.0833,74.7333,
45,Sorong,ID,84.69,scattered clouds,-0.8833,131.2500,
51,Banjar,ID,87.33,overcast clouds,-8.1900,114.9675,


In [7]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [8]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Port Moresby,PG,85.42,broken clouds,-9.4431,147.1797,Lamana Hotel (Port Moresby)
8,Hobyo,SO,83.79,light rain,5.3505,48.5268,Iftin hotel hobyo
15,Kapaa,US,84.18,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
24,Cayenne,GF,81.09,overcast clouds,4.9333,-52.3333,Hôtel Le Dronmi
30,Kathu,TH,85.82,overcast clouds,7.9178,98.3332,Novotel Phuket Resort


In [9]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [10]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [11]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))